<a href="https://colab.research.google.com/github/Herais/NLP_Learning_by_Selective_Data/blob/main/Open.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
####[1]####
# INTALL PROJECT FILES AND REQUIREMENTS
# Cells to run before this one: None
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Navigate to project directory
import os
if not os.path.exists('/content/drive/MyDrive/Github'):
  !mkdir Github
path_github   = '/content/drive/MyDrive/Github'
os.chdir(path_github)

# git clone
#!git clone https://github.com/Herais/NLP_Learning_by_Selective_Data.git
path_wd = path_github + '/' + 'NLP_Learning_by_Selective_Data'
os.chdir(path_wd)
!pwd

# INSTALL REQUIRED PACKAGES
!pip install -r requirements.txt
%tensorflow_version 1.x

Mounted at /content/drive
/content/drive/MyDrive/Github/NLP_Learning_by_Selective_Data
  Cloning https://www.github.com/bojone/bert4keras.git (to revision v0.8.3) to /tmp/pip-req-build-yzjamkmi
  Running command git clone -q https://www.github.com/bojone/bert4keras.git /tmp/pip-req-build-yzjamkmi
  Running command git checkout -q be69691e0ffdfd350d2d9fe501129a4b621d8a32
     |████████████████████████████████| 71kB 10.9MB/s 
     |████████████████████████████████| 563kB 35.5MB/s 
     |████████████████████████████████| 378kB 42.9MB/s 
     |████████████████████████████████| 51kB 7.9MB/s 
  Created wheel for folium: filename=folium-0.2.1-cp37-none-any.whl size=79979 sha256=b01b05ebd9a66494fd485ee01558b1823fdeb6d64b4a24bd5436afe24f9e151a
  Stored in directory: /root/.cache/pip/wheels/b8/09/f0/52d2ef419c2aaf4fb149f92a33e0008bdce7ae816f0dd8f0c5
  Created wheel for imgaug: filename=imgaug-0.2.5-cp37-none-any.whl size=561439 sha256=ff0cbf15ab0a343d1bcef90bdcee9306f51edcfc5f7e23daab55137cbee42

In [ ]:
####[2]####
# LOAD AND EXPLORE DATASETS
# Cells to run before this one: [1]
# Note: does not need GPU/TPU

# Import Libaries
import numpy as np
import pandas as pd

# Path
path_datasets ='/datasets'

# Load Dataset: Movie Title and Summary
path_dataset_movie = '/Movie_Title_and_Summary'
df_movie = pd.read_json(path_wd + path_datasets + path_dataset_movie + '/train.json', orient='records', lines=True)
df_movie.head(10)

# Load Dataset: Computer Research Literature (CSL) Title and Abstract
path_dataset_csl = '/CSL'
df_csl = pd.read_json(path_wd + path_datasets + path_dataset_csl + '/train_new_2500.json', orient='records', lines=True)
df_csl.head(10)

In [ ]:
####[3]####
# LOAD TRAINED MODELS
# Cells to run before this one: [1]
# [1] Mount Drive, Install Requirements

# Notes: Used load_weights rather than load_model here for simplicity to avoid defining customlayer,
# First reconstruct the model structure, then load trained weights.

# Load Libraries
from tensorflow import keras
from bert4keras.models import build_transformer_model

# Set Path
#path_models = '/models'
path_weights = '/weights'
path_pretrained_model = '/pretrained_model'

# Reconstruct Model Parameter to Load Weights
########################################
# Set Model Parameters
epochs = 10 # number of epochs to train
batch_size = 16 # 
maxlen = 256
topk = 1

# Set Pretrained Model Parameters (BERT)
config_path = path_wd + path_pretrained_model + '/chinese_L-12_H-768_A-12/bert_config.json'
checkpoint_path = path_wd + path_pretrained_model + '/chinese_L-12_H-768_A-12/bert_model.ckpt'
dict_path = path_wd + path_pretrained_model + '/chinese_L-12_H-768_A-12/vocab.txt'

# Load Vocab
from bert4keras.tokenizers import Tokenizer, load_vocab

token_dict, keep_tokens = load_vocab(
    dict_path=dict_path,
    simplified=True,
    startswith=['[PAD]', '[UNK]', '[CLS]', '[SEP]'],
)
tokenizer = Tokenizer(token_dict, do_lower_case=True)
########################################

# the Star Chaser Model trained with movie dataset only
model_StarChaser = build_transformer_model(
    config_path,
    checkpoint_path,
    application='unilm',
    keep_tokens=keep_tokens,  # include only tokens in keep tokens
)
path_model_weights = path_wd + path_weights + '/StarChaser_best_model.weights'
model_StarChaser.load_weights(path_model_weights)

# the Scholar Model trained with Computer Science Literature only
model_Scholar = build_transformer_model(
    config_path,
    checkpoint_path,
    application='unilm',
    keep_tokens=keep_tokens,   # include only tokens in keep tokens
)
path_model_weights = path_wd + path_weights + '/Scholar2_best_model.weights'
model_StarChaser.load_weights(path_model_weights)

# the Know-it-all Model Trained with both movie and cscl dataset
model_Knowitall = build_transformer_model(
    config_path,
    checkpoint_path,
    application='unilm',
    keep_tokens=keep_tokens,   # include only tokens in keep tokens
) 
path_model_weights = path_wd + path_weights + '/Knowitall2_best_model.weights'
model_StarChaser.load_weights(path_model_weights)


In [ ]:
####[4]####
# MAKE PREDICTIONS
# Cells to run before this one:  [1] and [3], or [1]+[5]+[6]
# [1] Mount Drive, Install Requirements
# [3] Load Models
# [5] Train Model - initialize model for trainning
# [6] Train Model - Training and save weights

# Load Library
import pandas as pd
import numpy as np
import json
from bert4keras.snippets import DataGenerator, AutoRegressiveDecoder

# Set Path
path_datasets ='/datasets'
path_predictions = '/predictions'
path_dataset_movie = '/Movie_Title_and_Summary'
path_dataset_csl = '/CSL'
path_test = path_wd + path_datasets + path_dataset_csl + '/test.json'
path_prediction = path_wd + path_predictions + '/model_Knowitall3_predicts_CSLTitles.json'

# Load Test Data
test_data = []
with open(path_test, encoding='utf-8') as f:
    for l in f:
        test_data.append(json.loads(l.strip()))

# Decoder: seq2seq
class AutoTitle(AutoRegressiveDecoder):
    #seq2seq decoder
    @AutoRegressiveDecoder.wraps(default_rtype='probas')
    def predict(self, inputs, output_ids, states):
        token_ids, segment_ids = inputs
        token_ids = np.concatenate([token_ids, output_ids], 1)
        segment_ids = np.concatenate([segment_ids, np.ones_like(output_ids)], 1)
        # CHANGE MODEL NAME HERE TO SWITCH PREDICTING MODELS:
        #####################
        return model.predict([token_ids, segment_ids])[:, -1]
        #####################

    def generate(self, text, topk=1):
        max_c_len = maxlen - self.maxlen
        token_ids, segment_ids = tokenizer.encode(text, maxlen=max_c_len)
        output_ids = self.beam_search([token_ids, segment_ids],
                                      topk=topk)  # beam search
        return tokenizer.decode(output_ids)

autotitle = AutoTitle(start_id=None, end_id=tokenizer._token_end_id, maxlen=32)

# Write to Prediction Path
with open(path_prediction ,'w') as f:
    for data in test_data:
        pred_title = ''.join(autotitle.generate(data["abst"], topk)).lower()
        output = {}
        output["id"] = data["id"]
        output["title"] = pred_title
        f.write(json.dumps(output, ensure_ascii = False))
        f.write('\n')

In [2]:
####[5]####
# TRAIN MORE MODELS - CONFIGURE
# Cells to run before this one: [1]
# [1] Mount Drive, Install Requirements

# Load Libraries
from __future__ import print_function
import numpy as np
import json
from tqdm import tqdm
from bert4keras.backend import keras, K
from bert4keras.layers import Loss
from bert4keras.models import build_transformer_model
from bert4keras.tokenizers import Tokenizer, load_vocab
from bert4keras.optimizers import Adam
from bert4keras.snippets import sequence_padding, open
from bert4keras.snippets import DataGenerator, AutoRegressiveDecoder
from keras.models import Model
from rouge import Rouge
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
import utils  # LocalLibraries

# Set Path
path_models = '/models'
path_datasets ='/datasets'
path_weights = '/weights'

# Set Name of this model
########################################
model_name = 'Knowitall3'
########################################

# Load Train and Dev Data
path_train = path_wd + path_datasets + '/CSL_Movie_Title_and_Summary/train.json'
path_dev = path_wd + path_datasets + '/CSL_Movie_Title_and_Summary/val.json'
train_data = utils.load_data(path_train)
valid_data = utils.load_data(path_dev)
# Could use pandas, but pandas has limitation on cell string length

# Set Model Parameter
epochs = 10
batch_size = 16
maxlen = 256
topk = 1

# Set BERT Path
# Download pretrained BERT model if necesary
path_pretrained_model = '/pretrained_model'
path_bert_chinese = path_wd + path_pretrained_model + '/chinese_L-12_H-768_A-12'
if not os.path.exists(path_bert_chinese):
 # Download Model
  !wget https://storage.googleapis.com/bert_models/2018_11_03/chinese_L-12_H-768_A-12.zip -P './pretrained_model/'
  %cd pretrained_model
  !unzip chinese_L-12_H-768_A-12.zip
  !rm -f chinese_L-12_H-768_A-12.zip
  os.chdir(path_wd)
  !pwd
config_path = path_wd + path_pretrained_model + '/chinese_L-12_H-768_A-12/bert_config.json'
checkpoint_path = path_wd + path_pretrained_model + '/chinese_L-12_H-768_A-12/bert_model.ckpt'
dict_path = path_wd + path_pretrained_model + '/chinese_L-12_H-768_A-12/vocab.txt'

# Load Vocab and Tokenize
token_dict, keep_tokens = load_vocab(
    dict_path=dict_path,
    simplified=True,
    startswith=['[PAD]', '[UNK]', '[CLS]', '[SEP]'],
)
tokenizer = Tokenizer(token_dict, do_lower_case=True)
 
# Data Generator
class data_generator(DataGenerator):
    """数据生成器
    """
    def __iter__(self, random=False):
        batch_token_ids, batch_segment_ids = [], []
        for is_end, (title, content) in self.sample(random):
            token_ids, segment_ids = tokenizer.encode(
                content, title, maxlen=maxlen
            )
            batch_token_ids.append(token_ids)
            batch_segment_ids.append(segment_ids)
            if len(batch_token_ids) == self.batch_size or is_end:
                batch_token_ids = sequence_padding(batch_token_ids)
                batch_segment_ids = sequence_padding(batch_segment_ids)
                yield [batch_token_ids, batch_segment_ids], None
                batch_token_ids, batch_segment_ids = [], []
train_generator = data_generator(train_data, batch_size)

# Loss Computation
class CrossEntropy(Loss):
    """交叉熵作为loss，并mask掉输入部分
    """
    def compute_loss(self, inputs, mask=None):
        y_true, y_mask, y_pred = inputs
        y_true = y_true[:, 1:]  # 目标token_ids
        y_mask = y_mask[:, 1:]  # segment_ids，刚好指示了要预测的部分
        y_pred = y_pred[:, :-1]  # 预测序列，错开一位
        loss = K.sparse_categorical_crossentropy(y_true, y_pred)
        loss = K.sum(loss * y_mask) / K.sum(y_mask)
        return loss

# Initialize Model with BERT
model = build_transformer_model(
    config_path,
    checkpoint_path,
    application='unilm',
    keep_tokens=keep_tokens,  # reduce vocab size
)
output = CrossEntropy(2)(model.inputs + model.outputs)

# Load Previous Weights for Continued Training
#path_model_weights = path_wd + path_weights + '/ScholarNew_best_model.weights'
#model.load_weights(path_model_weights)
######
model = Model(model.inputs, output)
model.compile(optimizer=Adam(1e-5))
model.summary()

Using TensorFlow backend.


Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
Input-Token (InputLayer)        (None, None)         0                                            
__________________________________________________________________________________________________
Input-Segment (InputLayer)      (None, None)         0                                            
__________________________________________________________________________________________________
Embedding-Token (Embedding)     multiple             10432512    Input-Token[0][0]                
                                                                 MLM-Norm[0][0]                   
__________________________________________________________________________________________________
Embedding-S

/tensorflow-1.15.2/python3.7/keras/engine/training_utils.py:819: UserWarning: Output cross_entropy_1 missing from loss dictionary. We assume this was done on purpose. The fit and evaluate APIs will not be expecting any data to be passed to cross_entropy_1.
  'be expecting any data to be passed to {0}.'.format(name))


In [3]:
####[6]####
# TRAIN MORE MODELS - TRAINING
# Cells to run before this one: [1],[4]
# [1] Mount Drive, Install Requirements
# [4] Train more models - config

# Decoder: seq2seq
class AutoTitle(AutoRegressiveDecoder):
    #seq2seq decoder
    @AutoRegressiveDecoder.wraps(default_rtype='probas')
    def predict(self, inputs, output_ids, states):
        token_ids, segment_ids = inputs
        token_ids = np.concatenate([token_ids, output_ids], 1)
        segment_ids = np.concatenate([segment_ids, np.ones_like(output_ids)], 1)
        # CHANGE MODEL NAME HERE TO SWITCH PREDICTING MODELS:
        return model.predict([token_ids, segment_ids])[:, -1]

    def generate(self, text, topk=1):
        max_c_len = maxlen - self.maxlen
        token_ids, segment_ids = tokenizer.encode(text, maxlen=max_c_len)
        output_ids = self.beam_search([token_ids, segment_ids],
                                      topk=topk)  # beam search
        return tokenizer.decode(output_ids)
autotitle = AutoTitle(start_id=None, end_id=tokenizer._token_end_id, maxlen=32)

# Callback
class Evaluator(keras.callbacks.Callback):
    def __init__(self, model_name=''):
        self.rouge = Rouge()
        self.smooth = SmoothingFunction().method1
        self.best_bleu = 0.
        self.valid_data = valid_data
        self.model_name = model_name # save weights given model name

    def on_epoch_end(self, epoch, logs=None):
        metrics = self.evaluate(valid_data)  # 评测模型
        if metrics['bleu'] > self.best_bleu:
            self.best_bleu = metrics['bleu']
            model.save_weights('./weights/' + self.model_name + '_best_model.weights')  # 保存模型
        metrics['best_bleu'] = self.best_bleu
        print('valid_data:', metrics)

    def evaluate(self, data, topk=1):
        total = 0
        rouge_1, rouge_2, rouge_l, bleu = 0, 0, 0, 0
        for title, content in tqdm(data):
            total += 1
            title = ' '.join(title).lower()
            pred_title = ' '.join(autotitle.generate(content, topk)).lower()
            if pred_title.strip():
                scores = self.rouge.get_scores(hyps=pred_title, refs=title)
                rouge_1 += scores[0]['rouge-1']['f']
                rouge_2 += scores[0]['rouge-2']['f']
                rouge_l += scores[0]['rouge-l']['f']
                bleu += sentence_bleu(
                    references=[title.split(' ')],
                    hypothesis=pred_title.split(' '),
                    smoothing_function=self.smooth
                )
        rouge_1 /= total
        rouge_2 /= total
        rouge_l /= total
        bleu /= total
        return {
            'rouge-1': rouge_1,
            'rouge-2': rouge_2,
            'rouge-l': rouge_l,
            'bleu': bleu,
        }


In [ ]:
####[6]####
# TRAIN MORE MODELS - TRAINING - Includes resetting path for continuous training
# Cells to run before this one: [1],[4]，[6]
# [1] Mount Drive, Install Requirements
# [4] Train more models - config
# [6] Run only once, then use this for continued training

# Load Train and Dev Data Here for Continued Training
########################################
#model_name = 'Knowitall'
########################################
#path_train = path_wd + path_datasets + '/Movie_Title_and_Summary/train.json'
#path_dev = path_wd + path_datasets + '/Movie_Title_and_Summary/val.json'
#train_data = utils.load_data(path_train)
#valid_data = utils.load_data(path_dev)

evaluator = Evaluator(model_name)
train_generator = data_generator(train_data, batch_size)

# Train Model
model.fit(
    train_generator.forfit(),
    steps_per_epoch=len(train_generator),
    epochs=epochs,
    callbacks=[evaluator]
)

# Save Trained Model
#utils.save_model_with_dt(path_wd + path_models, model, model_name)
# Alternatively best weights are saved under ./weights

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where

Epoch 1/10
1402/1402 [==============================] - 1149s 820ms/step - loss: 3.3779


100%|██████████| 5483/5483 [08:26<00:00, 10.82it/s]


valid_data: {'rouge-1': 0.24753894474090923, 'rouge-2': 0.17177220265852694, 'rouge-l': 0.24749489972439445, 'bleu': 0.1162587092141086, 'best_bleu': 0.1162587092141086}
Epoch 2/10
1180/1402 [========================>.....] - ETA: 3:02 - loss: 2.6018

In [ ]:
####[6]####
# LOAD PREDICTIONS
# Cells to run before this one: [1]
# [1] Mount Drive, Install Requirements
# Note: does not need GPU/TPU, for more analysis, ue Result.ipynb

# Load Results
path_predictions = '/predictions'
path_model_StarChaser_predicts_MovieTitles = '/model_StarChaser_predicts_MovieTitles.json'
path_model_StarChaser_predicts_CSLTitles = '/model_StarChaser_predicts_CSLTitles.json'
path_model_Scholar_predicts_CSLTitles = '/model_Scholar_predicts_CSLTitles.json'
path_model_Scholar_predicts_MovieTitles = '/model_Scholar_predicts_MovieTitles.json'
path_model_Knowitall_predicts_CSLTitles = '/model_Knowitall_predicts_CSLTitles.json'
path_model_Knowitall_predicts_MovieTitles = '/model_Knowitall_predicts_MovieTitles.json'